In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import copy

In [ ]:
path_coverage = 'tme_data/sets/v2_maze/coverage/2'
os.chdir(path_coverage)

In [ ]:
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        return f.read()

In [ ]:
def neg_size_check(matrix, size):
    """
    Check if matrix == size and return False if that is the case
    """
    result = True
    if(matrix.shape[0] == size):
        result = False
    return result

In [ ]:
def matrix_build(matrix, limit):
    """
    Stretch matrix to size: limit
    """
    result = matrix
    if (neg_size_check(matrix, limit)):
        number = limit-matrix.shape[0]
        if number < 0:
            result = matrix[:limit]
        else:
            temp_2 = np.full(number, matrix[-1])
            result = np.insert(result, -1, temp_2)
    return result

In [ ]:
def min_mean_max_std_extraction(matrix):
    """
    Extract min_mean_max from matrix with return [min, mean, max]
    """
    result = np.empty(4)
    result[0] = np.min(matrix)
    result[1] = np.mean(matrix)
    result[2] = np.max(matrix)
    result[3] = np.std(matrix)
    return result

In [ ]:
def min_median_max_extraction(matrix):
    """
    Extract min_median_max from matrix with return [min, median, max]
    """
    result = np.empty(3)
    result[0] = np.min(matrix)
    result[1] = np.median(matrix)
    result[2] = np.max(matrix)
    return result

In [ ]:
def matrix_splice(matrix, length):
    """
    Return matrices spliced together with length being the number of robots for experiment to be evaluated
    """
    result_distance = matrix[:, 0]
    result_count = matrix[:, 0]
    result_distance_sum = matrix[:, 0]
    result_count_sum = matrix[:, 0]

    temp = matrix[:, 1:]
    loop = temp.shape[1]
    loop = loop/length
    loop = int(loop)

    for i in range(loop):
        num_1 = length
        num_2 = length*2
        temp_distance = temp[:, 0:num_1]
        temp_count = temp[:, num_1:num_2]
        temp_distance_sum = np.sum(temp_distance, axis=1)
        temp_count_sum = np.sum(temp_count, axis=1)
        result_distance=np.c_[result_distance, temp_distance]
        result_count=np.c_[result_count, temp_count]
        result_distance_sum = np.c_[result_distance_sum, temp_distance_sum]
        result_count_sum = np.c_[result_count_sum, temp_count_sum]
        temp = temp[:, num_2:]
        
    return result_distance, result_count, result_distance_sum, result_count_sum

In [ ]:
def matrix_builder(matrix, distance_matrix, limit, map_name):
    """
    Returns coverage, travel distance and index of time if the requirements are met in terms of travel_distance and minimum coverage by robot
    """
    temp = []
    for i in range(limit):
        temp.append(i*5)
    temp = np.array(temp)
    result = copy.deepcopy(temp)
    result_2 = np.array(temp)
    result_2 = copy.deepcopy(temp)
    index_array = []
    for i in range(len(matrix)):
        temp = matrix[i][:, 1]
        temp = matrix_build(temp, limit)
        if(i < distance_matrix.shape[0]):
            temp_2 = distance_matrix[i][:,1:]
            temp_2 = eighty_build_distance(temp_2, limit)
            if (temp[-1] > 0.88 and np.all(temp_2[-1] > 10)):
                test_passed = False
                start_value = -1
                index = 0
                count = 0
                for j in range(limit - 1):
                    if map_name == 'marty':
                        if (temp[j+1] > 0.88 and index == 0):
                            index = j+1
                    else:
                        if (temp[j+1] > 0.88 and index == 0):
                            index = j+1
                    for x in range(len(temp_2[0])):
                        if (np.abs(temp_2[j][x] - temp_2[j+1][x]) > 5):
                            test_passed = True
                if(not test_passed and index != 0):
                    result = np.c_[result, temp]
                    result_2 = np.c_[result_2, temp_2]
                    index_array.append(index)
    index_array = np.array(index_array)
    return result, result_2, index_array

In [ ]:
def eighty_build_distance(matrix, limit):
    """
    Return distance matrix sized to limit by appending last value until limit is reached
    """
    result = matrix
    if (neg_size_check(matrix, limit)):
        number = limit-matrix.shape[0]
        if number < 0:
            result = matrix[:limit]
            number = 0
        #temp_2 = np.full((number, 4), matrix[-1][:])
        #temp_2 = np.repeat(matrix[-1], number, axis=0)
        temp_2 = np.tile(matrix[-1], (number, 1))
        result = np.append(result, temp_2)
        result = result.reshape((limit, -1))
    return result

In [ ]:
def extract_from_file(eval_method):
    result = []
    output = 0
    
    for file in sorted(os.listdir()):
        if file.endswith('.txt'):
            file_path =  f"{file}"
            if(eval_method in file_path):
                check = np.loadtxt(file_path)
                if check.any() and check.ndim == 2:
                    output += 1
                    #print(file)
                    #print(check.shape)
                    result.append(check)
    result = np.asarray(result)
    #if result.ndim > 1:
    #    print(result)
    return result

In [ ]:
def extraction(map_name, robot_num, eval_methods, limit=100, eval_lower=0, eval_upper=-1):
    try: 
        os.chdir(f'../../{map_name}/coverage/{robot_num}')
    except:
        os.chdir(f'../../../{map_name}/coverage/{robot_num}')

    matrix_coverage = []
    len_eval = len(eval_methods)

    for i in range(len_eval):
        testing = extract_from_file(eval_methods[i])
        matrix_coverage.append(testing)

    matrix_coverage = np.array(matrix_coverage)

    os.chdir(f'../../../{map_name}/distances/{robot_num}')

    matrix_distance = []
    for i in range(len_eval):
        matrix_distance.append(extract_from_file(eval_methods[i]))

    matrix_distance = np.array(matrix_distance)
        
    c_mean = np.empty((len_eval, limit))
    time = np.empty((len_eval, 4))
    c_sum = []
    d_sum = []

    for i in range(len(eval_methods)):
        print(f'robot:{robot_num} with {eval_methods[i]} on {map_name}')
        nearest_c_plot, nearest_d_plot, nearest_time_taken = matrix_builder(matrix_coverage[i], matrix_distance[i], 100, map_name)

        time_values_nearest = min_mean_max_std_extraction(nearest_time_taken[eval_lower:eval_upper])
    
        nearest_mean = np.mean(nearest_c_plot[:, eval_lower:eval_upper], axis=1)
        
        nearest_d, nearest_c, nearest_c_sum, nearest_d_sum = matrix_splice(nearest_d_plot, robot_num)

        nearest_c_sum_mean = np.mean(nearest_c_sum[:, 1:-1], axis=1)
        
        nearest_d_sum_mean = np.mean(nearest_d_sum[:, 1:-1], axis=1)
        
        c_mean[i] = nearest_mean
        time[i] = time_values_nearest
        c_sum.append(nearest_c_sum_mean)
        d_sum.append(nearest_d_sum_mean)

    c_sum = np.array(c_sum)
    d_sum = np.array(d_sum)

    return c_mean, time, c_sum, d_sum


In [ ]:
evals = ['nearest', 'minPos', 'nextFrontier', 'coExplore', 'co122']  # ['nearest', 'minPos', 'nextFrontier', 'coExplore', 'co122']
eval_length = len(evals)

map_2_a, map_2_b, map_2_c, map_2_d = extraction('map_3', 2, evals)
map_3_a, map_3_b, map_3_c, map_3_d = extraction('map_3', 3, evals)
map_4_a, map_4_b, map_4_c, map_4_d = extraction('map_3', 4, evals)
map_5_a, map_5_b, map_5_c, map_5_d = extraction('map_3', 5, evals)

maze_2_a, maze_2_b, maze_2_c, maze_2_d = extraction('v2_maze', 2, evals)
maze_3_a, maze_3_b, maze_3_c, maze_3_d = extraction('v2_maze', 3, evals)
maze_4_a, maze_4_b, maze_4_c, maze_4_d = extraction('v2_maze', 4, evals)
maze_5_a, maze_5_b, maze_5_c, maze_5_d = extraction('v2_maze', 5, evals)

marty_2_a, marty_2_b, marty_2_c, marty_2_d = extraction('marty', 2, evals)
marty_3_a, marty_3_b, marty_3_c, marty_3_d = extraction('marty', 3, evals)
marty_4_a, marty_4_b, marty_4_c, marty_4_d = extraction('marty', 4, evals)
marty_5_a, marty_5_b, marty_5_c, marty_5_d = extraction('marty', 5, evals) 
save = True

In [ ]:
os.chdir('../../../../..')

In [ ]:
map = np.empty((4, eval_length))
xdata = [2,3,4,5]
map[0] = map_2_b[:, 1]
map[1] = map_3_b[:, 1]
map[2] = map_4_b[:, 1]
map[3] = map_5_b[:, 1]
map = map * 5
fig = plt.figure()
plt.plot(xdata, map[:,0], label='nearest')
plt.plot(xdata, map[:,1], label='minPos')
plt.plot(xdata, map[:,2], label='nextFrontier')
plt.plot(xdata, map[:,3], label='coExplore')
plt.plot(xdata, map[:,4], label='co122')
plt.xlabel('Number of robots for experiment')
plt.ylabel('Mean exploration time in seconds')
plt.xticks(xdata)
plt.legend()
#plt.savefig("map_3_results.svg", transparent=True)

In [ ]:
maze = np.empty((4, eval_length))
maze[0] = maze_2_b[:, 1]
maze[1] = maze_3_b[:, 1]
maze[2] = maze_4_b[:, 1]
maze[3] = maze_5_b[:, 1]
maze = maze * 5
plt.plot(xdata, maze[:,0], label='nearest')
plt.plot(xdata, maze[:,1], label='minPos')
plt.plot(xdata, maze[:,2], label='nextFrontier')
plt.plot(xdata, maze[:,3], label='coExplore')
plt.plot(xdata, maze[:,4], label='co122')
plt.xticks(xdata)
plt.xlabel('Number of robots for experiment')
plt.ylabel('Mean exploration time in seconds')
plt.legend()
#plt.savefig("maze_results.svg", transparent=True)
plt.show()

In [ ]:
marty = np.empty((4, eval_length))
marty[0] = marty_2_b[:, 1]
marty[1] = marty_3_b[:, 1]
marty[2] = marty_4_b[:, 1]
marty[3] = marty_5_b[:, 1]
marty = marty * 5
plt.plot(xdata, marty[:,0], label='nearest')
plt.plot(xdata, marty[:,1], label='minPos')
plt.plot(xdata, marty[:,2], label='nextFrontier')
plt.plot(xdata, marty[:,3], label='coExplore')
plt.plot(xdata, marty[:,4], label='co122')
plt.xlabel('Number of robots for experiment')
plt.ylabel('Mean exploration time in seconds')
plt.xticks(xdata)
plt.legend()
#plt.savefig('marty_results.svg', transparent=True)

In [ ]:
marty

In [ ]:
maze

In [ ]:
map

In [ ]:
dist = np.loadtxt('tme_data/distances/5/v2_maze_coExplore_2022-03-04|07:33:48.txt')
cov = np.loadtxt('tme_data/coverage/measurements/5/v2_maze_coExplore_2022-03-04-07:33:48.txt')

In [ ]:
xdata = cov[:, 0]
print(xdata)
fig = plt.figure()
plt.plot(xdata, cov[:,1])
plt.ylabel('Exploration in %')
plt.xlabel('Exploration time in seconds')
plt.xticks(xdata[::2])
if save:
    plt.savefig("maze_video.png")

In [ ]:
xdata = cov[:, 0]
print(xdata)
fig = plt.figure()
plt.plot(xdata, cov[:,1], 'w')
plt.ylabel('Exploration in %')
plt.xlabel('Exploration time in seconds')
plt.xticks(xdata[::2])
if save:
    plt.savefig("maze_empty.png")

In [ ]:
dist_sum = np.sum(dist[:, 1:], axis=1)
dist_sum

In [ ]:
xdata = dist[:, 0]
fig = plt.figure()
plt.plot(xdata, dist_sum)
plt.ylabel('Exploration distance in meters')
plt.xlabel('Exploration time in seconds')
plt.xticks(xdata[::2])
if save:
    plt.savefig("maze_video_dist.png")

In [ ]:
xdata = dist[:, 0]
fig = plt.figure()
plt.plot(xdata, dist_sum, 'w')
plt.ylabel('Exploration distance in meters')
plt.xlabel('Exploration time in seconds')
plt.xticks(xdata[::2])
if save:
    plt.savefig("maze_video_dist_empty.png")

In [ ]:
xdata = [2,3,4,5]
maze = np.empty((4, eval_length))
maze[0] = maze_2_b[:, 1]
maze[1] = maze_3_b[:, 1]
maze[2] = maze_4_b[:, 1]
maze[3] = maze_5_b[:, 1]
maze2 = np.empty((4, eval_length))
maze2[0] = maze_2_b[:, 3]
maze2[1] = maze_3_b[:, 3]
maze2[2] = maze_4_b[:, 3]
maze2[3] = maze_5_b[:, 3]
maze = maze * 5
plt.errorbar(np.array(xdata)+0.2, maze[:,0], maze2[:,0], label='nearest', ls='None', marker='o')
plt.errorbar(np.array(xdata)+0.1, maze[:,1], maze2[:,1], label='minPos', ls='None', marker='o')
plt.errorbar(np.array(xdata), maze[:,2], maze2[:,2], label='nextFrontier', ls='None', marker='o')
plt.errorbar(np.array(xdata)-0.1, maze[:,3], maze2[:,3], label='coExplore', ls='None', marker='o')
plt.errorbar(np.array(xdata)-0.2, maze[:,4], maze2[:,4], label='co122', ls='None', marker='o')
plt.xticks(xdata)
plt.xlabel('Number of robots for experiment')
plt.ylabel('Mean exploration time in seconds')
plt.legend()
plt.axvline(x=2.5, color='black', linewidth=1)
plt.axvline(x=3.5, color='black', linewidth=1)
plt.axvline(x=4.5, color='black', linewidth=1)
if save:
    plt.savefig("maze_results.svg", transparent=True)
plt.show()

In [ ]:
xdata = [2,3,4,5]
marty = np.empty((4, eval_length))
marty[0] = marty_2_b[:, 1]
marty[1] = marty_3_b[:, 1]
marty[2] = marty_4_b[:, 1]
marty[3] = marty_5_b[:, 1]
marty2 = np.empty((4, eval_length))
marty2[0] = marty_2_b[:, 3]
marty2[1] = marty_3_b[:, 3]
marty2[2] = marty_4_b[:, 3]
marty2[3] = marty_5_b[:, 3]
marty = marty * 5
plt.errorbar(np.array(xdata)+0.2, marty[:,0], marty2[:,0], label='nearest')
plt.errorbar(np.array(xdata)+0.1, marty[:,1], marty2[:,1], label='minPos')
plt.errorbar(np.array(xdata), marty[:,2], marty2[:,2], label='nextFrontier')
plt.errorbar(np.array(xdata)-0.1, marty[:,3], marty2[:,3], label='coExplore')
plt.errorbar(np.array(xdata)-0.2, marty[:,4], marty2[:,4], label='co122')
plt.xticks(xdata)
plt.xlabel('Number of robots for experiment')
plt.ylabel('Mean exploration time in seconds')
plt.legend()
plt.fill_between(np.array(xdata)+0.2, marty[:,0]+marty2[:,0], marty[:,0]-marty2[:,0], alpha=0.25)
plt.fill_between(np.array(xdata)+0.1, marty[:,1]+marty2[:,1], marty[:,1]-marty2[:,1], alpha=0.25)
plt.fill_between(np.array(xdata), marty[:,2]+marty2[:,2], marty[:,2]-marty2[:,2], alpha=0.25)
plt.fill_between(np.array(xdata)-0.1, marty[:,3]+marty2[:,3], marty[:,3]-marty2[:,3], alpha=0.25)
plt.fill_between(np.array(xdata)-0.2, marty[:,4]+marty2[:,4], marty[:,4]-marty2[:,4], alpha=0.25)
if save:
    plt.savefig("marty_results.svg", transparent=True)
plt.show()

In [ ]:
xdata = [2,3,4,5]
marty = np.empty((4, eval_length))
marty[0] = marty_2_b[:, 1]
marty[1] = marty_3_b[:, 1]
marty[2] = marty_4_b[:, 1]
marty[3] = marty_5_b[:, 1]
marty2 = np.empty((4, eval_length))
marty2[0] = marty_2_b[:, 3]
marty2[1] = marty_3_b[:, 3]
marty2[2] = marty_4_b[:, 3]
marty2[3] = marty_5_b[:, 3]
marty = marty * 5
plt.errorbar(np.array(xdata)-0.2, marty[:,0], marty2[:,0], label='nearest', ls='None', marker='o')
plt.errorbar(np.array(xdata)-0.1, marty[:,1], marty2[:,1], label='minPos', ls='None', marker='o')
plt.errorbar(np.array(xdata), marty[:,2], marty2[:,2], label='nextFrontier', ls='None', marker='o')
plt.errorbar(np.array(xdata)+0.1, marty[:,3], marty2[:,3], label='coExplore', ls='None', marker='o')
plt.errorbar(np.array(xdata)+0.2, marty[:,4], marty2[:,4], label='co122', ls='None', marker='o')
plt.xticks(xdata)
plt.xlabel('Number of robots for experiment')
plt.ylabel('Mean exploration time in seconds')
plt.legend()
plt.axvline(x=2.5, color='black', linewidth=1)
plt.axvline(x=3.5, color='black', linewidth=1)
plt.axvline(x=4.5, color='black', linewidth=1)
if save:
    plt.savefig("marty_results.svg", transparent=True)
plt.show()

In [ ]:
xdata = [2,3,4,5]
map = np.empty((4, eval_length))
map[0] = map_2_b[:, 1]
map[1] = map_3_b[:, 1]
map[2] = map_4_b[:, 1]
map[3] = map_5_b[:, 1]
map2 = np.empty((4, eval_length))
map2[0] = map_2_b[:, 3]
map2[1] = map_3_b[:, 3]
map2[2] = map_4_b[:, 3]
map2[3] = map_5_b[:, 3]
map = map * 5
plt.errorbar(np.array(xdata)+0.2, map[:,0], map2[:,0], label='nearest', ls='None', marker='o')
plt.errorbar(np.array(xdata)+0.1, map[:,1], map2[:,1], label='minPos', ls='None', marker='o')
plt.errorbar(np.array(xdata), map[:,2], map2[:,2], label='nextFrontier', ls='None', marker='o')
plt.errorbar(np.array(xdata)-0.1, map[:,3], map2[:,3], label='coExplore', ls='None', marker='o')
plt.errorbar(np.array(xdata)-0.2, map[:,4], map2[:,4], label='co122', ls='None', marker='o')
plt.xticks(xdata)
plt.xlabel('Number of robots for experiment')
plt.ylabel('Mean exploration time in seconds')
plt.legend()
plt.axvline(x=2.5, color='black', linewidth=1)
plt.axvline(x=3.5, color='black', linewidth=1)
plt.axvline(x=4.5, color='black', linewidth=1)
if save:
    plt.savefig("map_results.svg", transparent=True)
plt.show()